In [61]:
class RegistroUsuario:
    def __init__(self, nome=None, endereco=None, telefone=None, email=""):
        self.dados = {"Nome" : nome,
                      "Endereço": endereco,
                      "Telefone": telefone,
                      "Email" : email}

    def recupera_campos(self):
        return self.dados

    def seta_campo(self, nomeCampo, valor):
        if nomeCampo in self.dados:
            self.dados[nomeCampo] = valor
        else:
            raise Exception(f"Campo {nomeCampo} inexistente")

In [62]:
import socket
import json
from threading import Thread
class ServidorAtendimento:
    def __init__(self, endereco_servidor="0.0.0.0", porta_servidor=3216, max_conexoes=1):
        # criação do socket e configuração
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.bind((endereco_servidor, porta_servidor))
        self.socket.listen(max_conexoes)

        # Registro de thread para atendimento e registros de usuários
        self.threadClientes = {}
        
        # Simbolo da moeda, valor -> quanto 1 BRL compra dessa moeda
        self.listaMoedas = {
                            "USD": 0.18,
                            "EUR": 0.16,
                            "GBP": 0.13,
                            "JPY": 28.50,
                            "AUD": 0.27,
                            "CAD": 0.24,
                            "CHF": 0.16,
                            "CNY": 1.27,
                            "ARS": 32.50,
                            }

        # Inicia uma thread dedicada para escuta de novas conexões
        self.threadEscuta = Thread(target=self.implementacaoThreadEscuta)
        self.threadEscuta.start()

    def handlerDeMensagem(self, mensagem):
        return mensagem

    def implementacaoThreadCliente(self, enderecoDoCliente, socketParaCliente):
        retries = 3
        socketParaCliente.settimeout(10) # timout de 10 segundos

        while True:
            try:
                mensagem = socketParaCliente.recv(512) # aguarda por comando
            except TimeoutError as e:
                print(f"Cliente {enderecoDoCliente} não enviou mensagens nos últimos 10 minutos. Encerrando a conexão")
                socketParaCliente.close() # fecha a conexão com o cliente pelo lado do servidor
                break # quebra o loop infinito e termina a thread
            except Exception as e:
                # caso o socket tenha a conexão fechada pelo cliente ou algum outro erro que não timeout
                print(f"Cliente {enderecoDoCliente} fechou a conexão com exceção: {e}")
                break

            # Se a mensagem for vazia, espere a próxima
            if len(mensagem) != 0:
                retries = 3
            else:
                retries -= 1
                if retries == 0:
                    break
                continue


            print(f"Servidor recebeu do cliente {enderecoDoCliente} a mensagem: {json.loads(mensagem.decode('utf-8'))}")

            # Decodifica mensagem em bytes para utf-8 e
            # em seguida decodifica a mensagem em Json para um dicionário Python
            mensagem_decodificada = json.loads(mensagem.decode("utf-8"))
            print(mensagem_decodificada)
            # Por enquanto, retorna a mensagem recebida
            resposta = self.handlerDeMensagem(mensagem_decodificada)
            print(resposta)
            # fim do while
            resposta_bytes = json.dumps(resposta).encode("utf-8")

            print(f"Servidor enviou para o cliente {enderecoDoCliente} a mensagem: {resposta}")

            socketParaCliente.send(resposta_bytes)

        # Testaremos apenas com um usuário por servidor
        # Forçaremos a parada da thread de escuta fechando socket
        self.socket.close()

    def implementacaoThreadEscuta(self):
        while True:
            # Thread fica bloqueada enquanto aguarda por conexões,
            # enquanto servidor continua rodando normalmente
            try:
                (socketParaCliente, enderecoDoCliente) = self.socket.accept()
            except OSError:
                # Como fechamos o socket na thread para cliente,
                # quando tentarmos escutar no mesmo socket, ele não mais
                # existirá e lançará um erro
                # Não é isso que servidores de verdade fazem, é só um exemplo
                print(f"Servidor: desligando thread de escuta")
                break
            self.threadClientes[enderecoDoCliente] = Thread(target=self.implementacaoThreadCliente,
                                                            args=(enderecoDoCliente, socketParaCliente),
                                                            daemon=True) # thread sem necessidade de join, será morta ao final do processo
            self.threadClientes[enderecoDoCliente].run() # inicia thread de atendimento ao novo cliente conectado

In [63]:
import time

def cliente():
    # Recupera endereço do servidor
    socket_cliente_thread = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    nome_servidor = socket.gethostname()
    print(nome_servidor)
    ip_servidor = socket.gethostbyname_ex(nome_servidor)
    print(ip_servidor)

    # coloca a thread para dormir por dois segundos enquanto o servidor é iniciado
    time.sleep(2)

    # conecta com o servidor
    socket_cliente_thread.connect((ip_servidor[2][0], 3216))

    mensagem = {"SERVIÇO" : "Dados Pessoais"}
    # Transforma dicionário em JSON e em seguida para bytes
    mensagem_bytes = json.dumps(mensagem).encode("utf-8")

    # envia mensagem ao servidor
    socket_cliente_thread.send(mensagem_bytes)
    msg = socket_cliente_thread.recv(512)
    print("Cliente:", msg)
    socket_cliente_thread.close()
    print("Cliente:", socket_cliente_thread)

In [64]:
# Cria uma thread cliente
from concurrent.futures import ThreadPoolExecutor
threadPool = ThreadPoolExecutor()
threadPool.submit(cliente)

# Cria o servidor
servidor = ServidorAtendimento()
del servidor

LAPTOP-DVTH613D
('LAPTOP-DVTH613D.localdomain', ['LAPTOP-DVTH613D'], ['127.0.1.1'])


Servidor recebeu do cliente ('127.0.0.1', 38640) a mensagem: {'SERVIÇO': 'Dados Pessoais'}
{'SERVIÇO': 'Dados Pessoais'}
{'SERVIÇO': 'Dados Pessoais'}
Servidor enviou para o cliente ('127.0.0.1', 38640) a mensagem: {'SERVIÇO': 'Dados Pessoais'}
Cliente: b'{"SERVI\\u00c7O": "Dados Pessoais"}'
Cliente: <socket.socket [closed] fd=-1, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>
Servidor: desligando thread de escuta


Utilizaremos as seguintes mensagens JSON

* Para listar as moedas disponíveis para conversão

```json
{
    "ACAO": "LISTAR MOEDAS",
    "MOEDA": ""
}
```

```json
{
    "ACAO": "CONSULTAR MOEDA",
    "MOEDA": ""
}
```
O servidor retornará uma mensagem JSON no formato:
```json
{
    "STATUS CODE": "",
    "RESPONSE": "",
}
```

In [65]:
def requisicoesHandler(self, mensagem_decodificada):
    resposta = {}
    while True:
        if "AÇÃO" not in mensagem_decodificada:
            resposta = {"STATUS CODE": "404",
                        "RESPONSE" : "Mensagem não contém uma entrada de AÇÃO"
                        }
            break 
        if mensagem_decodificada["AÇÃO"] not in ["CONSULTAR MOEDAS", "LISTAR MOEDAS"]:
            resposta = {"STATUS CODE": "404",
                        "RESPONSE" : f"Mensagem contém uma entrada inválida de AÇÃO:{mensagem_decodificada['AÇÃO']}"
                        }
            break 
        
        if mensagem_decodificada["AÇÃO"] == "LISTAR MOEDAS":
            resposta = {"STATUS CODE": "200",
                        "RESPONSE" : f"Moedas disponíveis para conversão: {self.listaMoedas.keys()}"
                        }

    return resposta

ServidorAtendimento.handlerDeMensagem = requisicoesHandler

In [66]:
def cliente():
    # Recupera endereço do servidor
    socket_cliente_thread = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    nome_servidor = socket.gethostname()
    ip_servidor = socket.gethostbyname_ex(nome_servidor)
    print(ip_servidor)

    # coloca a thread para dormir por dois segundos enquanto o servidor é iniciado
    time.sleep(2)

    # conecta com o servidor
    socket_cliente_thread.connect((ip_servidor[2][0], 3216))
    
    mensagens = [
        {
            "AÇÃO": "LISTAR MOEDAS"
        }
    ]

    for mensagem in mensagens:
        # Transforma dicionário em JSON e em seguida para bytes
        mensagem_bytes = json.dumps(mensagem).encode("utf-8")

        # envia mensagem ao servidor
        socket_cliente_thread.send(mensagem_bytes)
        msg = socket_cliente_thread.recv(512)
        print("Cliente:", json.loads(msg.decode("utf-8")))
    socket_cliente_thread.close()
    print("Cliente:", socket_cliente_thread)

In [ ]:
# Cria uma thread cliente
from concurrent.futures import ThreadPoolExecutor
threadPool = ThreadPoolExecutor()
threadPool.submit(cliente)

# Cria o servidor
servidor = ServidorAtendimento()
del servidor

('LAPTOP-DVTH613D.localdomain', ['LAPTOP-DVTH613D'], ['127.0.1.1'])


Servidor recebeu do cliente ('127.0.0.1', 52374) a mensagem: {'AÇÃO': 'LISTAR MOEDAS'}
{'AÇÃO': 'LISTAR MOEDAS'}
